In [87]:
import pandas as pd
from nltk import word_tokenize
from string import punctuation
punctuation += '--'

In [88]:
def sp(word, G, segment_size=20):
    sp = 0
    segments = []
    for i in range(int(len(G)/segment_size)):
        segment = G[segment_size*i:segment_size*(i+1)]
        segments.append(segment)
    for segment in segments:
        if word in segment:
            sp += 1
    sp = sp / len(segments)
    return sp

In [41]:
def zeta(word, G1, G2, segment_size=100):
    return sp(word, G1, segment_size=segment_size) - sp(word, G2, segment_size=segment_size)

In [44]:
tweet_texts = pd.read_csv('data/trumptweets-1515775693.tweets.csv')['text']

In [170]:
people = ['@BarackObama',
          '@MittRomney',
          '@VPBiden',
          '@foxandfriends']

In [179]:
mentions = ['Putin',
           'Iraq',
           'Russia']

In [180]:
tweets_dict = {}

for tweet in tweet_texts:
    if any([person in tweet for person in people]):
        ref_person = people[[person in tweet for person in people].index(True)]
        tweet_words = [word for word in word_tokenize(tweet) if word not in punctuation and '@'+word not in people]
        if ref_person not in tweets_dict:
            tweets_dict[ref_person] = tweet_words
        else:
            tweets_dict[ref_person].extend(tweet_words)
    if any([word in tweet for word in mentions]):
        mention = mentions[[word in tweet for word in mentions].index(True)]
        tweet_words = [word for word in word_tokenize(tweet) if word not in punctuation
                       and '@'+word not in people
                       and word not in mentions]
        if mention not in tweets_dict:
            tweets_dict[mention] = tweet_words
        else:
            tweets_dict[mention].extend(tweet_words)

In [187]:
tweets_dict['@BarackObama'][7]

'China'

In [182]:
words = []

for person in tweets_dict:
    words.extend(tweets_dict[person])

In [183]:
def zetas(opposed):
    assert len(opposed) == 2
    
    zetas_dict = {}
    for word in set(words):
        G1 = tweets_dict[opposed[0]]
        G2 = tweets_dict[opposed[1]]
        zetas_dict[word] = zeta(word, G1, G2)
    return zetas_dict

In [184]:
def oppose(opposed, wordcount=20):
    assert len(opposed) == 2
    
    zetas_dict = zetas(opposed)
    zetas_set = {k: v for k, v in sorted(zetas_dict.items(), key=lambda item: item[1])}
    words_set = list(zetas_set.keys())
    
    print(opposed[0].upper())
    for word in words_set[:-wordcount:-1]:
        print(f'{word}: {zetas_dict[word]}')
    print()
    print(opposed[1].upper())
    for word in words_set[:wordcount]:
        print(f'{word}: {zetas_dict[word]}')

In [185]:
oppose(['@BarackObama', 'Putin'])

@BARACKOBAMA
http: 0.6604395604395604
cont: 0.4461538461538462
's: 0.29780219780219785
debt: 0.27692307692307694
jobs: 0.26153846153846155
in: 0.2362637362637363
Why: 0.2230769230769231
China: 0.21868131868131868
his: 0.2142857142857143
US: 0.2076923076923077
are: 0.20109890109890113
budget: 0.2
over: 0.19560439560439563
why: 0.19230769230769232
has: 0.18681318681318687
discussing: 0.18461538461538463
country: 0.17692307692307693
campaign: 0.17692307692307693
spending: 0.17692307692307693

PUTIN
President: -0.734065934065934
Obama: -0.6725274725274725
Vladimir: -0.5
you: -0.4637362637362637
with: -0.4494505494505494
realDonaldTrump: -0.4054945054945055
Trump: -0.3978021978021978
be: -0.3934065934065934
or: -0.3285714285714285
I: -0.321978021978022
think: -0.31868131868131866
him: -0.30549450549450546
'': -0.2835164835164835
world: -0.2802197802197802
``: -0.27802197802197803
Russian: -0.2626373626373626
at: -0.25384615384615383
bad: -0.24725274725274723
look: -0.24725274725274723
’: -0

In [186]:
oppose(['@BarackObama', '@MittRomney'])

@BARACKOBAMA
has: 0.26538461538461544
debt: 0.2519230769230769
http: 0.24615384615384617
in: 0.20769230769230773
our: 0.20192307692307693
cont: 0.19615384615384618
jobs: 0.18653846153846154
spending: 0.17692307692307693
his: 0.175
he: 0.16923076923076918
is: 0.16346153846153844
China: 0.16153846153846152
budget: 0.15000000000000002
Keystone: 0.14615384615384616
over: 0.13846153846153847
n't: 0.13846153846153847
gas: 0.13653846153846155
US: 0.13269230769230772
Now: 0.11923076923076924

@MITTROMNEY
My: -0.31923076923076926
interview: -0.3173076923076923
discussing: -0.3153846153846154
I: -0.28269230769230774
great: -0.2711538461538462
amp: -0.25
SquawkCNBC: -0.19423076923076923
'': -0.1942307692307692
Obama: -0.19038461538461537
night: -0.18846153846153846
Mitt: -0.18653846153846154
Great: -0.17692307692307693
needs: -0.17692307692307693
polls: -0.175
election: -0.1692307692307692
do: -0.16153846153846152
get: -0.15961538461538463
who: -0.1576923076923077
win: -0.15384615384615385
debate